In [1]:
# !git clone "https://github.com/SriRamGovardhanam/wastedata-Mask_RCNN-multiple-classes.git"


In [2]:
#!rm -rf /content/mrcnn

In [3]:
# import shutil, os
# def copytree(src = 'wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN', dst = '', symlinks=False, ignore=None):
#     try:
#         shutil.rmtree('/.ipynb_checkpoints')
#     except:
#         pass
#     for item in os.listdir(src):
#         s = os.path.join(src, item)
#         d = os.path.join(dst, item)
#         if os.path.isdir(s):
#             shutil.copytree(s, d, symlinks, ignore)
#         else:
#             shutil.copy2(s, d)
# copytree()
# os.makedirs("MaskRCNN/newdataset",exist_ok=True)
# shutil.copytree('MaskRCNN/newdataset','dataset')

In [4]:
#!pip install keras==2.2.5
#%tensorflow_version 1.x

In [5]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt
# Root directory of the project
ROOT_DIR = os.path.abspath("wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils
# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Directory to save logs and model checkpoints
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

2023-03-17 10:14:20.186403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 10:14:20.742600: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nouman/anaconda3/envs/MRCNN/lib/python3.9/site-packages/cv2/../../lib64:
2023-03-17 10:14:20.742627: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-17 10:14:22.220467: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
import os
import json
import numpy as np
import skimage.draw
from mrcnn.utils import Dataset

class CatDogDataset(Dataset):
    def load_dataset(self, dataset_dir, is_train=True):
        # Add classes
        self.add_class("catdog", 1, "cat")
        self.add_class("catdog", 2, "dog")

        # Load annotations from JSON file
        with open(os.path.join(dataset_dir, "annot/MRCNN_coco.json")) as f:
            annotations = json.load(f)

        # Get a list of all image files
        images_dir = dataset_dir + "/images"
        image_files = sorted(os.listdir(images_dir))
        image_ids = [i for i in range(len(image_files))]

        # Set train or validation dataset
        if is_train:
            self.train_ids = image_ids
        else:
            self.val_ids = image_ids

        # Add images and annotations
        for i, image_id in enumerate(image_ids):
            # Add image file
            image_file = image_files[image_id]
            image_path = images_dir + image_file
            self.add_image("catdog", image_id=image_id, path=image_path)

            # Add annotation
            image_annotations = annotations.get(image_file, [])
            for annotation in image_annotations:
                class_name = annotation['class_name']
                if class_name not in self.class_info:
                    continue
                class_id = self.class_names.index(class_name)
                mask = self.generate_mask(annotation)
                if mask is not None:
                    self.add_mask("catdog", image_id=image_id, class_id=class_id, mask=mask)



In [7]:
#pip install -r requirements.txt

In [8]:
import os
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import utils
#from custom_dataset import CustomDataset
import tensorflow.compat.v1 as tf
# Define the configuration for the model
class MyConfig(Config):
    # Give the configuration a recognizable name
    NAME = "myconfig"
    # Adjust the number of classes based on your custom dataset
    NUM_CLASSES = 2 + 1  # background + your classes
    # Other configuration parameters
    BACKBONE = "resnet50"
    IMAGES_PER_GPU = 2
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 50

# Load the pre-trained COCO weights into the model
COCO_MODEL_PATH = "/home/nouman/Downloads/training_batch/week5/mask_rcnn_coco.h5"
model = modellib.MaskRCNN(mode="training", config=MyConfig(), model_dir='./')
model.load_weights("/home/nouman/Downloads/training_batch/week5/mask_rcnn_coco.h5", by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

# # Replace the output layer of the model with a new output layer that matches the number of classes in your custom dataset
# model.layers[-1].kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)
# model.layers[-1].bias_initializer = tf.keras.initializers.Zeros()
# model.layers[-1].units = MyConfig.NUM_CLASSES
# model.layers[-1].activation = "sigmoid"

# # Freeze the layers of the pre-trained model up to the last few layers, leaving the new output layer unfrozen
# layers_to_freeze = 100
# for i, layer in enumerate(model.layers):
#     if i < layers_to_freeze:
#         layer.trainable = False
#     else:
#         layer.trainable = True

# Load the training and validation data
dataset_train = CatDogDataset()
dataset_train.load_dataset("/home/nouman/Downloads/training_batch/week5/catdog")
dataset_train.prepare()

#dataset_val = CustomDataset()
#dataset_val.load_dataset("path/to/validation/data", "val")
#dataset_val.prepare()

# Train the model on your custom dataset


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Instructions for updating:
box_ind is deprecated, use box_indices instead


2023-03-17 10:14:31.040283: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nouman/anaconda3/envs/MRCNN/lib/python3.9/site-packages/cv2/../../lib64:
2023-03-17 10:14:31.040312: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-17 10:14:31.040334: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nouman-HP-EliteBook-840-G2): /proc/driver/nvidia/version does not exist
2023-03-17 10:14:31.041040: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFl

In [10]:
model.train(dataset_train,dataset_train, epochs=10, layers="head",learning_rate=0.001)


Starting at epoch 0. LR=0.001

Checkpoint Path: ./myconfig20230317T1014/mask_rcnn_myconfig_{epoch:04d}.h5
Selecting layers to train


2023-03-17 10:14:46.903136: W tensorflow/c/c_api.cc:291] Operation '{name:'range_1' id:7163 op device:{requested: '', assigned: ''} def:{{{node range_1}} = Range[Tidx=DT_INT32, _has_manual_control_dependencies=true](range_1/start, Rank_1, range_1/delta)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


TypeError: unhashable type: 'ListWrapper'

In [8]:
# annotations1 = json.load(open(os.path.join("/home/nouman/Downloads/training_batch/week5/dataset/train", "cat_dog_annotations.json")))
#         # print(annotations1)
# annotations = list(annotations1.values())
# annotations[0].keys()

In [9]:
# annotations[1].keys()

In [10]:
# annotations[1]["1.jpeg45602"]["regions"]

In [11]:
# annotations1 = json.load(open(os.path.join("/home/nouman/Downloads/training_batch/week5/dataset/train", "cat_dog_annotations.json")))
#         # print(annotations1)
# annotation = list(annotations1.values())  # don't need the dict keys

# # The VIA tool saves images in the JSON even if they don't have any
# # annotations. Skip unannotated images.
# annotations= []
# for key in annotation[1].keys():
#     annot = annotation[1][key]["regions"]
#     annotations.append(annot)
# #annotations = [a for a in annotations if a['regions']]

        

In [43]:
# for i,x in enumerate(annotation[1].keys()):
#     print(annotation[1][x]["filename"])

1.jpeg
2.jpeg
3.jpeg
4.jpeg
5.jpeg
6.jpeg
7.jpeg


In [13]:
# # Add images
# for i,a in enumerate(annotations):
#             # print(a)
#             # Get the x, y coordinaets of points of the polygons that make up
#             # the outline of each object instance. There are stores in the
#             # shape_attributes (see json format above)
#             polygons = [r['shape_attributes'] for r in annotations[i]] 
#             objects = [s['region_attributes']['class'] for s in annotations[i]]
#             print("objects:",objects)
#             name_dict = {"cat": 1,"dog": 2} #,"xyz": 3}
#             # key = tuple(name_dict)
#             num_ids = [name_dict[a] for a in objects[i].keys()]
     
#             # num_ids = [int(n['Event']) for n in objects]
#             # load_mask() needs the image size to convert polygons to masks.
#             # Unfortunately, VIA doesn't include it in JSON, so we must read
#             # the image. This is only managable since the dataset is tiny.
#             print("numids",num_ids)
#             image_path = os.path.join(dataset_dir, a['filename'])
#             image = skimage.io.imread(image_path)
#             height, width = image.shape[:2]

#             self.add_image(
#                 "object",  ## for a single class just add the name here
#                 image_id=a['filename'],  # use file name as a unique image id
#                 path=image_path,
#                 width=width, height=height,
#                 polygons=polygons,
#                 num_ids=num_ids
#                 )

In [15]:
# for a in name_dict.keys():
#     print(a)

In [17]:
#a in objects

In [19]:
#name_dict

In [52]:
# class CustomDataset(utils.Dataset):

#     def load_custom(self, dataset_dir, subset):
#         """Load a subset of the Horse-Man dataset.
#         dataset_dir: Root directory of the dataset.
#         subset: Subset to load: train or val
#         """
#         # Add classes. We have only one class to add.
#         self.add_class("object", 1, "cat")
#         self.add_class("object", 2, "dog")
#         # self.add_class("object", 3, "xyz") #likewise

#         # Train or validation dataset?
#         assert subset in ["train", "val"]
#         dataset_dir = os.path.join(dataset_dir, subset)

#         # Load annotations
#         # VGG Image Annotator saves each image in the form:
#         # { 'filename': '28503151_5b5b7ec140_b.jpg',
#         #   'regions': {
#         #       '0': {
#         #           'region_attributes': {},
#         #           'shape_attributes': {
#         #               'all_points_x': [...],
#         #               'all_points_y': [...],
#         #               'name': 'polygon'}},
#         #       ... more regions ...
#         #   },
#         #   'size': 100202
#         # }
#         # We mostly care about the x and y coordinates of each region
#         annotations1 = json.load(open(os.path.join(dataset_dir, "cat_dog_annotations.json")))
#         # print(annotations1)
#         annotation = list(annotations1.values())  # don't need the dict keys

#         # The VIA tool saves images in the JSON even if they don't have any
#         # annotations. Skip unannotated images.
#         annotations= []
#         for key in annotation[1].keys():
#             annot = annotation[1][key]["regions"]
#             annotations.append(annot)
#         #annotations = [a for a in annotations if a['regions']]
        
#         # Add images
#         for i,a in enumerate(annotations):
#             # print(a)
#             # Get the x, y coordinaets of points of the polygons that make up
#             # the outline of each object instance. There are stores in the
#             # shape_attributes (see json format above)
#             polygons = [r['shape_attributes'] for r in annotations[i]] 
#             objects = [s['region_attributes']['class'] for s in annotations[i]]
#             print("objects:",objects)
#             name_dict = {"cat": 1,"dog": 2} #,"xyz": 3}
#             # key = tuple(name_dict)
#             num_ids = [name_dict[a] for a in objects[i].keys()]
     
#             num_ids = [int(n['Event']) for n in objects]
#             # load_mask() needs the image size to convert polygons to masks.
#             # Unfortunately, VIA doesn't include it in JSON, so we must read
#             # the image. This is only managable since the dataset is tiny.
#             #print("numids",num_ids)
#             image_path = os.path.join(dataset_dir, annotation[1][x]["filename"])
#             image = skimage.io.imread(image_path)
#             height, width = image.shape[:2]

#             self.add_image(
#                 "object",  ## for a single class just add the name here
#                 image_id=annotation[1][x]["filename"],  # use file name as a unique image id
#                 path=image_path,
#                 width=width, height=height,
#                 polygons=polygons,
#                 num_ids=num_ids
#                 )

#     def load_mask(self, image_id):
#         """Generate instance masks for an image.
#        Returns:
#         masks: A bool array of shape [height, width, instance count] with
#             one mask per instance.
#         class_ids: a 1D array of class IDs of the instance masks.
#         """
#         # If not a Horse/Man dataset image, delegate to parent class.
#         image_info = self.image_info[image_id]
#         if image_info["source"] != "object":
#             return super(self.__class__, self).load_mask(image_id)

#         # Convert polygons to a bitmap mask of shape
#         # [height, width, instance_count]
#         info = self.image_info[image_id]
#         if info["source"] != "object":
#             return super(self.__class__, self).load_mask(image_id)
#         num_ids = info['num_ids']
#         mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
#                         dtype=np.uint8)
#         for i, p in enumerate(info["polygons"]):
#             # Get indexes of pixels inside the polygon and set them to 1
#         	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

#         	mask[rr, cc, i] = 1

#         # Return mask, and array of class IDs of each instance. Since we have
#         # one class ID only, we return an array of 1s
#         # Map class names to class IDs.
#         num_ids = np.array(num_ids, dtype=np.int32)
#         return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

#     def image_reference(self, image_id):
#         """Return the path of the image."""
#         info = self.image_info[image_id]
#         if info["source"] == "object":
#             return info["path"]
#         else:
#             super(self.__class__, self).image_reference(image_id)

In [53]:
# def train(model):
#     """Train the model."""
#     # Training dataset.
#     dataset_train = CustomDataset()
#     dataset_train.load_custom("dataset", "train")
#     dataset_train.prepare()

#     # Validation dataset
#     dataset_val = CustomDataset()
#     dataset_val.load_custom("dataset", "val")
#     dataset_val.prepare()

#     # *** This training schedule is an example. Update to your needs ***
#     # Since we're using a very small dataset, and starting from
#     # COCO trained weights, we don't need to train too long. Also,
#     # no need to train all layers, just the heads should do it.
#     print("Training network heads")
#     model.train(dataset_train, dataset_val,
#                 learning_rate=config.LEARNING_RATE,
#                 epochs=10,
#                 layers='heads')

In [54]:
# DEFAULT_LOGS_DIR

'/home/nouman/Downloads/training_batch/week5/wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/logs'

In [55]:
# import tensorflow
# print(tensorflow.__version__)

2.11.0


In [56]:
# config = CustomConfig()
# model = modellib.MaskRCNN(mode="training", config=config,
#                                   model_dir=DEFAULT_LOGS_DIR)



In [57]:
# weights_path = COCO_WEIGHTS_PATH
#         # Download weights file
# if not os.path.exists(weights_path):
#     utils.download_trained_weights(weights_path)

# model.load_weights(weights_path, by_name=True, exclude=[
#             "mrcnn_class_logits", "mrcnn_bbox_fc",
#             "mrcnn_bbox", "mrcnn_mask"])

In [58]:
# train(model)


objects: [{'dog': True}]
numids [2]
objects: [{'dog': True}, {'cat': True}]
numids [1]
objects: [{'cat': True}]


IndexError: list index out of range

In [ ]:
# import os
# import sys
# import random
# import math
# import re
# import time
# import numpy as np
# import tensorflow as tf
# import matplotlib
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# import matplotlib.image as mpimg
# # Root directory of the project
# #ROOT_DIR = os.path.abspath("/")

# # Import Mask RCNN
# sys.path.append(ROOT_DIR)  # To find local version of the library
# from mrcnn import utils
# from mrcnn import visualize
# from mrcnn.visualize import display_images
# import mrcnn.model as modellib
# from mrcnn.model import log
# %matplotlib inline
# # Directory to save logs and trained model
# MODEL_DIR = os.path.join(ROOT_DIR, "logs")
# # Path to Ballon trained weights
# # You can download this file from the Releases page
# # https://github.com/matterport/Mask_RCNN/releases
# WEIGHTS_PATH = "/content/wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/logs/object20201209T0658/mask_rcnn_object_0010.h5"  # TODO: update this path

In [21]:
# config = CustomConfig()
# CUSTOM_DIR = os.path.join(ROOT_DIR, "/content/dataset/")
# class InferenceConfig(config.__class__):
#   # Run detection on one image at a time
#   GPU_COUNT = 1
#   IMAGES_PER_GPU = 1
#   DETECTION_MIN_CONFIDENCE = 0.7
# config = InferenceConfig()
# config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [22]:
# # Device to load the neural network on. Useful if you're training a model on the same machine, in which case use CPU and leave the GPU for training.
# DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
# # Inspect the model in training or inference modes values: 'inference' or 'training'
# TEST_MODE = "inference"
# def get_ax(rows=1, cols=1, size=16):
#   '''Return a Matplotlib Axes array to be used in all visualizations in the notebook. Provide a central point to control graph sizes. Adjust the size attribute to control 
#   how big to render images'''
#   _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
#   return ax

# # Load validation dataset
# CUSTOM_DIR = "/content/dataset"
# dataset = CustomDataset()
# dataset.load_custom(CUSTOM_DIR, "val")
# # Must call before using the dataset
# dataset.prepare()
# print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset/val/cat_dog_annotations.json'

In [ ]:
# #LOAD MODEL
# # Create model in inference mode
# with tf.device(DEVICE):
#   model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

In [ ]:
# # Load COCO weights Or, load the last model you trained
# weights_path = WEIGHTS_PATH
# # Load weights
# print("Loading weights ", weights_path)
# model.load_weights(weights_path, by_name=True)

In [ ]:
# #RUN DETECTION
# image_id = random.choice(dataset.image_ids)
# print(image_id)
# image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#   modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
# info = dataset.image_info[image_id]
# print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id,dataset.image_reference(image_id)))
# # Run object detection
# results = model.detect([image], verbose=1)
# # Display results
# x = get_ax(1)
# r = results[0]
# visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], dataset.class_names, r['scores'], ax=ax, title="Predictions")
# log("gt_class_id", gt_class_id)
# log("gt_bbox", gt_bbox)
# log("gt_mask", gt_mask)
# # This is for predicting images which are not present in dataset
# path_to_new_image = '/content/dataset/test/unnamed.jpg'
# image1 = mpimg.imread(path_to_new_image)
# # Run object detection
# print(len([image1]))
# results1 = model.detect([image1], verbose=1)
# # Display results
# ax = get_ax(1)
# r1 = results1[0]
# visualize.display_instances(image1, r1['rois'], r1['masks'], r1['class_ids'],
# dataset.class_names, r1['scores'], ax=ax, title="Predictions1")